In [1]:
from ola_models import * 
from ola_RNN import * 

import os, time, copy, math, re, json, pickle, random
import numpy as np
import pandas as pd

import torch, torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

from functools import partial 

cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_available else "cpu")
print(f'''using device {device}''')

path = !pwd
path = path[0]
print(path)

using device cuda:0
/home/r2/Documents/RNNexp


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [11]:
bs  = 30
sql = 30 
lr  = 0.0005 

sched = combine_scheds([0.15, 0.25, 0.2, 0.4], [sched_cos(0.0005, 0.0008), sched_cos(0.0008, 0.0005),sched_lin(0.0005,0.0005),sched_cos(0.0005,0.00005)]) 
data          = pp_trumpdata(path+"/data/trump/", [0.9,0.95], bs)
data.train_dl = TweetDataLoader(data,data.train.tweets,bs,sql,shuffle=True)
data.valid_dl = TweetDataLoader(data,data.valid.tweets,bs,sql,shuffle=False)

model  = cuda(RNN(len(data.decoder), 150, len(data.decoder)))
opt    = optim.RMSprop(model.parameters(), lr, weight_decay=0.005, momentum=0.9)
learn  = Learner(model,  nn.NLLLoss(), opt , data, lr=lr)

In [4]:

    
grad_dict = {} 
def fun_hook(layer_name, grad_input, grad_output):
    print(grad_output)

def fc_hook(layer_name, grad_input, grad_output): 
    if layer_name in grad_dict:
        grad_dict[layer_name]["grad_input"].append(grad_input)
        grad_dict[layer_name]["grad_output"].append(grad_output)
    else :
        grad_dict[layer_name] = {}
        grad_dict[layer_name]["grad_input"] = []
        grad_dict[layer_name]["grad_output"] = []

In [111]:
# for p in model.parameters():
#     p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))    
    
class HookCallback(Callback):
    _order = 15

    def begin_fit(self,learn):
        super().begin_fit(learn)
        learn.stats.grads = []         
        for p in model.parameters():
            p.register_hook(lambda grad: learn.stats.grads.append(grad.mean().item()))
        return True

    def after_loss(self,loss):
        return True
    
    def after_step(self):
        return True
        

In [112]:
cbs = CallbackHandler([CounterCallback(4),StatsCallback(),HookCallback()])
# learn.model.register_backward_hook(fun_hook)

In [113]:
model  = cuda(RNN(len(data.decoder), 150, len(data.decoder)))
opt    = optim.RMSprop(model.parameters(), lr, weight_decay=0.005, momentum=0.9)
learn  = Learner(model,  nn.NLLLoss(), opt , data, lr=lr)

In [114]:
fit_rnn(1,learn,cbs)

learn.n_iters shouldn't be 0


In [115]:
i = 0 
for g in learn.stats.grads:
    i+=1
print(i)

64


In [148]:
def get_grads(learn):
    layers = 0
    for p in learn.model.parameters():
        layers+=1
    tmp = []
    for i in range(0,len(learn.stats.grads),layers):
        tmp.append(np.array(learn.stats.grads[i:i+layers]).reshape(layers,1))
    return np.hstack(tmp)

In [149]:
grad_means = get_grads(learn)

In [150]:
grad_means.shape

(16, 4)